# RPG Legacy Dependency Analyzer: Finding Islands in Your Codebase

## What This Notebook Does

This tool helps you **understand the structure of legacy RPG/SQL codebases** by:

1. **Parsing** source code to extract dependencies (which programs call which programs, which tables they access)
2. **Loading** this information into a Neo4j graph database
3. **Analyzing** the graph to find "islands" - isolated clusters of code that don't interact with other parts of the system
4. **Using AI** to automatically generate descriptive names and summaries for each island

## Why This Matters

**Finding islands helps with:**
- **Modernization Planning**: Identify self-contained subsystems that can be migrated independently
- **Risk Assessment**: Understand which changes might have ripple effects across the system
- **Documentation**: Auto-generate dependency maps for systems with outdated documentation
- **Team Organization**: Assign different islands to different teams

## How It Works

This is a **complete step-by-step tutorial** that takes you from raw RPG source code all the way to AI-powered island analysis:

```
Source Code → Parser → Graph Database → Clustering Algorithm → Islands → AI Analysis → Descriptions
```

The process follows these phases:

- **Phase 1**: Environment Setup - Configure connections and verify dependencies
- **Phase 2**: Parsing Source Code - Extract program calls and table accesses from RPG files
- **Phase 3**: Loading Into Neo4j - Create graph nodes and relationships in Neo4j
- **Phase 4**: Finding Islands with Graph Data Science - Use WCC algorithm to identify isolated clusters
- **Phase 5**: Enriching the Graph - Add island metadata and complexity metrics
- **Phase 6**: Verification and Exploration - Query and visualize the results
- **Phase 7**: AI-Powered Island Analysis - Use DeepSeek API to analyze source code and generate descriptive names and summaries for each island

Let's get started!

---
## Phase 1: Environment Setup

Before we begin, we need to:
1. Import required Python libraries
2. Configure connection settings
3. Verify our environment is ready

### Step 1.1: Import Libraries

We'll use:
- `os` and `re` for file operations and pattern matching
- `pandas` for data manipulation
- `neo4j` for graph database connectivity

In [1]:
import os
import re
import pandas as pd
from neo4j import GraphDatabase

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


### Step 1.2: Configure Paths and Connections

Set up:
- **REPO_PATH**: Where your RPG source files are located
- **NEO4J_URI**: Connection string to your Neo4j database
- **NEO4J_AUTH**: Username and password for Neo4j

> **Note**: If running in the provided DevContainer, these defaults should work as-is.

In [2]:
# Configuration
REPO_PATH = "./src"  # Folder containing your .rpgle, .sqlrpgle, .clp files
NEO4J_URI = "bolt://localhost:7687"  # Database connection
NEO4J_AUTH = ("neo4j", "password")  # Default credentials

print("✅ Configuration loaded")
print(f"   Repository: {REPO_PATH}")
print(f"   Database: {NEO4J_URI}")

✅ Configuration loaded
   Repository: ./src
   Database: bolt://localhost:7687


### Step 1.3: Verify Environment

Check that the source code directory exists and is accessible.

In [3]:
# Verify source directory
abs_path = os.path.abspath(REPO_PATH)
print(f"📂 Scanning Directory: {abs_path}")

if not os.path.exists(REPO_PATH):
    os.makedirs(REPO_PATH)
    print(f"⚠️  Directory '{REPO_PATH}' was created. Please add your RPG source files there!")
else:
    # Count files
    file_count = sum(1 for root, dirs, files in os.walk(REPO_PATH) 
                     for f in files if f.lower().endswith(('.rpgle', '.sqlrpgle', '.rpg', '.clp', '.clle')))
    print(f"✅ Found {file_count} RPG source files")
    
    if file_count == 0:
        print("⚠️  No RPG files detected. Make sure you've added source code to the directory.")

📂 Scanning Directory: /workspace/src
✅ Found 364 RPG source files


---
## Phase 2: Parsing Source Code

Now we'll extract dependencies from the RPG source code.

### Understanding What We're Looking For

The parser identifies two types of dependencies:

1. **Program-to-Program Calls** (`CALLS` relationship)
   - `CALL`, `CALLP`, `CALLB` operations
   
2. **Program-to-Table Accesses** (`ACCESSES` relationship)
   - File declarations (`DCL-F`, F-specs)
   - Database operations (`CHAIN`, `READ`, `WRITE`, `UPDATE`, `DELETE`)
   - Embedded SQL (`SELECT`, `INSERT`, `UPDATE` statements)

### Why RPG Parsing is Complex: A Brief History

RPG (Report Program Generator) has evolved significantly since the 1960s:

- **RPG I/II (1960s-1970s)**: Fixed-format, column-dependent syntax (characters in specific columns had specific meanings)
- **RPG III (1980s)**: Added structured programming concepts, still fixed-format
- **RPG IV / ILE RPG (1990s)**: Introduced free-format support with `/FREE` directives, but many programs kept fixed-format for compatibility
- **Fully Free RPG (2000s+)**: Modern `**FREE` syntax, resembling contemporary languages

**The Challenge:** Most legacy systems contain **all these formats mixed together** - some files are purely fixed-format, others are fully free, and many use **mixed-mode** (fixed-format sections with `/FREE` blocks embedded). A robust parser must handle all variations, understand column-based rules for old code, and modern syntax for new code - often within the same file!

### Parser Features

Our parser handles all RPG generations:
- ✅ **Free-format RPG** (`**FREE` directive - modern style)
- ✅ **Fixed-format RPG** (column-based - legacy style)
- ✅ **Mixed-mode** (`/FREE` ... `/END-FREE` blocks within fixed-format files)
- ✅ **Embedded SQL** (`EXEC SQL` statements - can appear in any format)
- ✅ **Line number tracking** (for traceability and debugging)

### Step 2.1: Define Parser Helper Functions

These functions handle string literals and SQL parsing.

In [4]:
# ============================================================================
# MAIN PARSER FUNCTION
# ============================================================================

def parse_rpg_file(filepath):
    """
    Parse a single RPG source file and extract all dependencies.
    
    This is the main orchestrator that uses all helper and detection functions.
    
    Args:
        filepath: Path to the RPG source file
        
    Returns:
        list: List of dependency dictionaries, each containing:
            - source: The program name
            - source_path: Relative path to source file
            - source_ext: File extension
            - target: What it calls/accesses
            - type: 'CALLS' or 'ACCESSES'
            - action: 'EXECUTE', 'READ', 'WRITE', 'UPDATE', 'SQL'
            - line: Source line number
            - statement: The actual source line
    """
    dependencies = []
    
    # Extract file metadata
    full_filename = os.path.basename(filepath)
    filename = full_filename.split('.')[0].upper()
    extension = full_filename.split('.')[-1].upper() if '.' in full_filename else ''
    
    clean_path = os.path.relpath(filepath, start=".")
    if clean_path.startswith("./"): 
        clean_path = clean_path[2:]
    
    # Read file
    try:
        with open(filepath, 'r', encoding='utf-8', errors='replace') as f:
            lines = f.readlines()
    except Exception as e:
        print(f"❌ Error reading {filepath}: {e}")
        return []
    
    # Detect format mode
    is_fully_free = False
    if lines and len(lines) > 0:
        if lines[0].strip().upper().startswith('**FREE'):
            is_fully_free = True
    
    # State tracking
    in_free_block = False
    is_sql_block = False
    accumulated_sql_string = ""
    sql_string_start_line = 0
    
    # Process each line
    for line_num, raw_line in enumerate(lines, 1):
        line = raw_line.strip()
        if not line:
            continue
        
        upper_line = line.upper()
        
        # Handle /FREE and /END-FREE directives
        if upper_line.startswith('/FREE') or upper_line.startswith('//FREE'):
            in_free_block = True
            continue
        
        if upper_line.startswith('/END-FREE') or upper_line.startswith('//END-FREE'):
            in_free_block = False
            continue
        
        is_free_context = is_fully_free or in_free_block
        
        # Strip comments
        if not is_free_context:
            if len(raw_line) > 6 and raw_line[6] == '*':
                continue
        
        if '//' in line:
            line = line.split('//')[0].strip()
        if '--' in line:
            line = line.split('--')[0].strip()
        if not line:
            continue
        
        # Create metadata for this line
        source_meta = {
            'source': filename,
            'source_path': clean_path,
            'source_ext': extension,
            'line': line_num,
            'statement': line
        }
        
        # Extract string literals and create cleaned version
        line_for_matching, string_literals = extract_and_strip_string_literals(line)
        upper_line_for_matching = line_for_matching.upper()
        
        # ====================================================================
        # CHECK FOR DYNAMIC SQL IN STRING LITERALS
        # ====================================================================
        for sql_string in string_literals:
            if is_sql_string(sql_string):
                # Handle multi-line SQL strings with + continuation
                if accumulated_sql_string:
                    accumulated_sql_string += " " + sql_string
                else:
                    accumulated_sql_string = sql_string
                    sql_string_start_line = line_num
                
                # Check if line continues
                if '+' not in line or line.rstrip().endswith(';'):
                    # String is complete, extract tables
                    tables = extract_tables_from_sql(accumulated_sql_string)
                    for table in tables:
                        item = source_meta.copy()
                        item['line'] = sql_string_start_line
                        item.update({'target': table, 'type': 'ACCESSES', 'action': 'SQL'})
                        dependencies.append(item)
                    
                    # Reset accumulator
                    accumulated_sql_string = ""
                    sql_string_start_line = 0
        
        # ====================================================================
        # PATTERN DETECTION (using cleaned line without string literals)
        # ====================================================================
        
        # A. DCL-F (Free-format file declaration)
        result = detect_dcl_f(line_for_matching, upper_line, is_free_context)
        if result:
            item = source_meta.copy()
            item.update(result)
            dependencies.append(item)
        
        # B. F-Spec (Fixed-format file specification)
        result = detect_f_spec(line, is_free_context)
        if result:
            item = source_meta.copy()
            item.update(result)
            dependencies.append(item)
        
        # C. Database OpCodes (CHAIN, READ, WRITE, etc.)
        result = detect_opcode(line_for_matching)
        if result:
            item = source_meta.copy()
            item.update(result)
            dependencies.append(item)
        
        # D. Program Calls (CALL, CALLP, CALLB)
        result = detect_call(line_for_matching)
        if result:
            item = source_meta.copy()
            item.update(result)
            dependencies.append(item)
        
        # E. Embedded SQL (EXEC SQL blocks)
        if 'EXEC SQL' in upper_line:
            is_sql_block = True
        if ';' in line:
            is_sql_block = False
        
        if is_sql_block or 'EXEC SQL' in upper_line:
            result = detect_embedded_sql_table(line_for_matching)
            if result:
                item = source_meta.copy()
                item.update(result)
                dependencies.append(item)
    
    return dependencies


print("✅ Main parser function defined")

✅ Main parser function defined


In [5]:
# ============================================================================
# HELPER FUNCTIONS: String Literals and SQL Parsing
# ============================================================================

def extract_and_strip_string_literals(text):
    """
    Extract string literals from text, then remove them for safe pattern matching.
    
    Args:
        text: Source code line
        
    Returns:
        tuple: (cleaned_text, list_of_string_contents)
        
    Example:
        >>> extract_and_strip_string_literals("dsply 'Hello' '' world;")
        ("dsply '' '' world;", ['Hello', ''])
    """
    string_contents = []
    
    # Extract single-quoted strings (handle escaped quotes '')
    for match in re.finditer(r"'([^']*(?:''[^']*)*)'", text):
        content = match.group(1).replace("''", "'")  # Unescape doubled quotes
        string_contents.append(content)
    
    # Extract double-quoted strings
    for match in re.finditer(r'"([^"]*)"', text):
        string_contents.append(match.group(1))
    
    # Remove string literals from text (replace with empty strings)
    cleaned = re.sub(r"'[^']*(?:''[^']*)*'", "''", text)
    cleaned = re.sub(r'"[^"]*"', '""', cleaned)
    
    return cleaned, string_contents


def extract_tables_from_sql(sql_string):
    """
    Extract table names from a SQL statement string.
    
    Args:
        sql_string: SQL statement (e.g., "SELECT * FROM EMPLOYEE WHERE id = 1")
        
    Returns:
        list: Table names found in the SQL
        
    Example:
        >>> extract_tables_from_sql("SELECT * FROM CORPDATA.EMPLOYEE")
        ['EMPLOYEE']
    """
    tables = []
    
    # SQL patterns that reference tables
    patterns = [
        r'\bFROM\s+(\w+(?:\.\w+)?)',           # SELECT ... FROM table
        r'\bJOIN\s+(\w+(?:\.\w+)?)',           # ... JOIN table
        r'\bINTO\s+(\w+(?:\.\w+)?)',           # INSERT INTO table
        r'\bUPDATE\s+(\w+(?:\.\w+)?)',         # UPDATE table
        r'\bINSERT\s+INTO\s+(\w+(?:\.\w+)?)',  # INSERT INTO table
        r'\bDELETE\s+FROM\s+(\w+(?:\.\w+)?)',  # DELETE FROM table
    ]
    
    for pattern in patterns:
        for match in re.finditer(pattern, sql_string, re.IGNORECASE):
            table_ref = match.group(1)
            # Handle schema.table format - extract just table name
            table_name = table_ref.split('.')[-1].upper()
            if table_name and table_name not in tables:
                tables.append(table_name)
    
    return tables


def is_sql_string(text):
    """
    Check if a string contains SQL keywords (indicating dynamic SQL).
    
    Args:
        text: String to check
        
    Returns:
        bool: True if string appears to contain SQL
    """
    sql_keywords = ['SELECT', 'INSERT', 'UPDATE', 'DELETE', 'FROM', 'INTO', 'JOIN']
    text_upper = text.upper()
    return any(keyword in text_upper for keyword in sql_keywords)


print("✅ Helper functions defined (string literals & SQL parsing)")

✅ Helper functions defined (string literals & SQL parsing)


In [6]:
def parse_rpg_file(filepath):
    """
    Parse a single RPG source file and extract dependencies.
    
    Returns:
        List of dictionaries, each containing:
        - source: The program name
        - target: What it calls/accesses
        - type: 'CALLS' or 'ACCESSES'
        - action: 'EXECUTE', 'READ', 'WRITE', 'UPDATE', 'SQL'
        - line: Source line number
    """
    
    def extract_and_strip_string_literals(text):
        """Extract string literals, then remove them for safe pattern matching.
        Returns: (cleaned_text, list_of_string_contents)
        """
        string_contents = []
        
        # Extract single-quoted strings
        for match in re.finditer(r"'([^']*(?:''[^']*)*)'", text):
            content = match.group(1).replace("''", "'")  # Unescape doubled quotes
            string_contents.append(content)
        
        # Extract double-quoted strings
        for match in re.finditer(r'"([^"]*)"', text):
            string_contents.append(match.group(1))
        
        # Remove string literals from text
        cleaned = re.sub(r"'[^']*(?:''[^']*)*'", "''", text)
        cleaned = re.sub(r'"[^"]*"', '""', cleaned)
        
        return cleaned, string_contents
    
    def extract_tables_from_sql(sql_string):
        """Extract table names from a SQL string."""
        tables = []
        sql_upper = sql_string.upper()
        
        # Look for FROM, JOIN, INTO, UPDATE, INSERT INTO, DELETE FROM patterns
        patterns = [
            r'\bFROM\s+(\w+(?:\.\w+)?)',
            r'\bJOIN\s+(\w+(?:\.\w+)?)',
            r'\bINTO\s+(\w+(?:\.\w+)?)',
            r'\bUPDATE\s+(\w+(?:\.\w+)?)',
            r'\bINSERT\s+INTO\s+(\w+(?:\.\w+)?)',
            r'\bDELETE\s+FROM\s+(\w+(?:\.\w+)?)',
        ]
        
        for pattern in patterns:
            for match in re.finditer(pattern, sql_upper):
                table_ref = match.group(1)
                # If schema.table format, take just the table name
                table_name = table_ref.split('.')[-1]
                if table_name and table_name not in tables:
                    tables.append(table_name)
        
        return tables
    
    dependencies = []
    full_filename = os.path.basename(filepath)
    filename = full_filename.split('.')[0].upper()
    extension = full_filename.split('.')[-1].upper() if '.' in full_filename else ''
    
    clean_path = os.path.relpath(filepath, start=".")
    if clean_path.startswith("./"): 
        clean_path = clean_path[2:]
    
    # Read file
    try:
        with open(filepath, 'r', encoding='utf-8', errors='replace') as f:
            lines = f.readlines()
    except Exception as e:
        print(f"❌ Error reading {filepath}: {e}")
        return []
        
    # Detect format mode
    is_fully_free = False
    if lines and len(lines) > 0:
        if lines[0].strip().upper().startswith('**FREE'):
            is_fully_free = True

    # State tracking for mixed-mode files
    in_free_block = False 
    is_sql_block = False
    accumulated_sql_string = ""  # For multi-line SQL strings
    sql_string_start_line = 0
    
    for line_num, raw_line in enumerate(lines, 1):
        line = raw_line.strip()
        if not line: 
            continue
        
        upper_line = line.upper()

        # Handle /FREE and /END-FREE directives
        if upper_line.startswith('/FREE') or upper_line.startswith('//FREE'):
            in_free_block = True
            continue
        
        if upper_line.startswith('/END-FREE') or upper_line.startswith('//END-FREE'):
            in_free_block = False
            continue

        is_free_context = is_fully_free or in_free_block

        # Strip comments
        if not is_free_context:
            if len(raw_line) > 6 and raw_line[6] == '*': 
                continue
        
        if '//' in line: 
            line = line.split('//')[0].strip()
        if '--' in line: 
            line = line.split('--')[0].strip()
        if not line: 
            continue

        # Metadata for this dependency
        source_meta = {
            'source': filename,
            'source_path': clean_path,
            'source_ext': extension,
            'line': line_num,
            'statement': line
        }
        
        # === Extract string literals and create cleaned version ===
        line_for_matching, string_literals = extract_and_strip_string_literals(line)
        upper_line_for_matching = line_for_matching.upper()
        
        # === Check for SQL in string literals (Dynamic SQL) ===
        for sql_string in string_literals:
            sql_upper = sql_string.upper()
            # Check if this string contains SQL keywords
            if any(keyword in sql_upper for keyword in ['SELECT', 'INSERT', 'UPDATE', 'DELETE', 'FROM', 'INTO']):
                # Handle multi-line SQL strings with + continuation
                if accumulated_sql_string:
                    accumulated_sql_string += " " + sql_string
                else:
                    accumulated_sql_string = sql_string
                    sql_string_start_line = line_num
                
                # Check if line continues (ends with + or has + continuation)
                if '+' not in line or line.rstrip().endswith(';'):
                    # String is complete, extract tables
                    tables = extract_tables_from_sql(accumulated_sql_string)
                    for table in tables:
                        item = source_meta.copy()
                        item['line'] = sql_string_start_line  # Use the line where SQL started
                        item.update({'target': table, 'type': 'ACCESSES', 'action': 'SQL'})
                        dependencies.append(item)
                    
                    # Reset accumulator
                    accumulated_sql_string = ""
                    sql_string_start_line = 0

        # === DETECTION PATTERNS (using cleaned line) ===
        
        # A. Modern DCL-F (Free Format File Declaration)
        if is_free_context:
            dcl_match = re.search(r'\bDCL-F\s+(\w+)', line_for_matching, re.IGNORECASE)
            if dcl_match:
                target = dcl_match.group(1).upper()
                action = 'READ' 
                if 'USAGE(*OUTPUT' in upper_line: action = 'WRITE'
                elif 'USAGE(*UPDATE' in upper_line: action = 'UPDATE'
                elif 'USAGE(*INPUT' in upper_line: action = 'READ'
                
                item = source_meta.copy()
                item.update({'target': target, 'type': 'ACCESSES', 'action': action})
                dependencies.append(item)

        # B. Legacy F-Specs (Fixed Format)
        if not is_free_context and upper_line.startswith('F') and len(line) >= 17:
            file_type = line[16].upper()
            if file_type in ['I', 'O', 'U', 'C']:
                target = line[6:16].strip().upper()
                if '=' not in target and '(' not in target:
                    action = 'WRITE' if file_type == 'O' else 'READ'
                    if file_type == 'U': action = 'UPDATE'
                    if file_type == 'C': action = 'READ/WRITE'
                    
                    item = source_meta.copy()
                    item.update({'target': target, 'type': 'ACCESSES', 'action': action})
                    dependencies.append(item)

        # C. Database OpCodes (using cleaned line to avoid string literal matches)
        opcode_match = re.search(r'\b(CHAIN|READ|READE|READP|WRITE|UPDAT|DELETE)\s+(\w+)', line_for_matching, re.IGNORECASE)
        if opcode_match:
            op = opcode_match.group(1).upper()
            target = opcode_match.group(2).upper()
            action = 'WRITE' if op in ['WRITE', 'UPDAT', 'DELETE'] else 'READ'
            
            item = source_meta.copy()
            item.update({'target': target, 'type': 'ACCESSES', 'action': action})
            dependencies.append(item)

        # D. Program Calls (using cleaned line)
        call_match = re.search(r'\b(CALL|CALLB|CALLP)\s+[\'\" ]?(\w+)[\'\" ]?', line_for_matching, re.IGNORECASE)
        if call_match:
            target = call_match.group(2).upper()
            item = source_meta.copy()
            item.update({'target': target, 'type': 'CALLS', 'action': 'EXECUTE'})
            dependencies.append(item)

        # E. Embedded SQL (EXEC SQL blocks)
        if 'EXEC SQL' in upper_line: 
            is_sql_block = True
        if ';' in line: 
            is_sql_block = False
        
        if is_sql_block or 'EXEC SQL' in upper_line:
            sql_match = re.search(r'\b(FROM|JOIN|INTO|UPDATE|INSERT INTO)\s+(\w+)', line_for_matching, re.IGNORECASE)
            if sql_match:
                raw_target = sql_match.group(2).upper()
                target = raw_target.split('.')[-1]
                
                item = source_meta.copy()
                item.update({'target': target, 'type': 'ACCESSES', 'action': 'SQL'})
                dependencies.append(item)

    return dependencies

print("✅ Parser function defined")

✅ Parser function defined


### Step 2.2: Scan All Source Files

Now we'll walk through the source directory and parse every RPG file.

In [7]:
# Supported file extensions
ALLOWED_EXTENSIONS = ('.rpgle', '.sqlrpgle', '.rpg', '.clp', '.clle')

all_deps = []
files_scanned = 0

print("🔍 Scanning source files...")

for root, dirs, files in os.walk(REPO_PATH):
    for file in files:
        if file.lower().endswith(ALLOWED_EXTENSIONS):
            files_scanned += 1
            filepath = os.path.join(root, file)
            all_deps.extend(parse_rpg_file(filepath))

print(f"\n✅ Scan complete!")
print(f"   Files processed: {files_scanned}")
print(f"   Dependencies found: {len(all_deps)}")

🔍 Scanning source files...

✅ Scan complete!
   Files processed: 364
   Dependencies found: 168


### Step 2.3: Convert to DataFrame

We'll convert the raw dependency list into a pandas DataFrame for easier analysis.

In [8]:
df_deps = pd.DataFrame(all_deps)

if not df_deps.empty:
    print(f"✅ Created DataFrame with {len(df_deps)} rows")
    print(f"\nColumns: {', '.join(df_deps.columns)}")
else:
    print("⚠️  No dependencies found. Check that your source files contain parseable RPG code.")

✅ Created DataFrame with 168 rows

Columns: source, source_path, source_ext, line, statement, target, type, action


### Step 2.4: Examine Sample Dependencies

Let's look at a few examples of what we found.

In [9]:
if not df_deps.empty:
    print("📋 First 10 dependencies:\n")
    display(df_deps.head(10))
else:
    print("No data to display")

📋 First 10 dependencies:



,source,source_path,source_ext,line,statement,target,type,action
0,LOADCUST2,src/IBM-i-RPG-Free-CLP-Code/5250_Subfile/LOADC...,CLLE,19,SUBMIT: SBMJOB CMD(CALL PGM(LOADCUSTR)...,PGM,CALLS,EXECUTE
1,LOADCUSTR,src/IBM-i-RPG-Free-CLP-Code/5250_Subfile/LOADC...,SQLRPGLE,102,from lennons1.csz,LENNONS1,ACCESSES,SQL
2,LOADCUSTR,src/IBM-i-RPG-Free-CLP-Code/5250_Subfile/LOADC...,SQLRPGLE,112,from lennons1.csz,LENNONS1,ACCESSES,SQL
3,LOADCUSTR,src/IBM-i-RPG-Free-CLP-Code/5250_Subfile/LOADC...,SQLRPGLE,120,exec sql fetch from csz_cur into :csz;,CSZ_CUR,ACCESSES,SQL
4,LOADCUSTR,src/IBM-i-RPG-Free-CLP-Code/5250_Subfile/LOADC...,SQLRPGLE,203,exec sql insert into lennons1.custmast values(...,LENNONS1,ACCESSES,SQL
5,LOADCUSTR,src/IBM-i-RPG-Free-CLP-Code/5250_Subfile/LOADC...,SQLRPGLE,205,SQLProblem('Insert into custmast');,CUSTMAST,ACCESSES,SQL
6,MTNCUSTR,src/IBM-i-RPG-Free-CLP-Code/5250_Subfile/MTNCU...,SQLRPGLE,51,dcl-f MTNCUSTD workstn infds(dfInfDS) indds(df...,MTNCUSTD,ACCESSES,READ
7,MTNCUSTR,src/IBM-i-RPG-Free-CLP-Code/5250_Subfile/MTNCU...,SQLRPGLE,170,write SH_HDR;,SH_HDR,ACCESSES,WRITE
8,MTNCUSTR,src/IBM-i-RPG-Free-CLP-Code/5250_Subfile/MTNCU...,SQLRPGLE,331,from CUSTMAST,CUSTMAST,ACCESSES,SQL
9,MTNCUSTR,src/IBM-i-RPG-Free-CLP-Code/5250_Subfile/MTNCU...,SQLRPGLE,490,from STATES,STATES,ACCESSES,SQL


### Step 2.5: Dependency Statistics

Let's get a statistical overview of what we found.

In [10]:
if not df_deps.empty:
    print("📊 Dependency Breakdown:\n")
    
    # By type
    print("By Type:")
    print(df_deps['type'].value_counts())
    
    print("\nBy Action:")
    print(df_deps['action'].value_counts())
    
    print("\nBy File Extension:")
    print(df_deps['source_ext'].value_counts())
    
    # Unique counts
    print(f"\n📌 Unique Programs: {df_deps['source'].nunique()}")
    print(f"📌 Unique Targets: {df_deps['target'].nunique()}")
else:
    print("No data to analyze")

📊 Dependency Breakdown:

By Type:
type
ACCESSES    133
CALLS        35
Name: count, dtype: int64

By Action:
action
WRITE      49
READ       44
SQL        38
EXECUTE    35
UPDATE      2
Name: count, dtype: int64

By File Extension:
source_ext
RPGLE       74
SQLRPGLE    64
CLLE        23
CLP          7
Name: count, dtype: int64

📌 Unique Programs: 50
📌 Unique Targets: 51


---
## Phase 3: Loading Into Neo4j

Now we'll transform our flat dependency list into a **graph structure** in Neo4j.

### Why Neo4j?

Graph databases excel at:
- **Relationship queries**: "What does Program X call?" becomes trivial
- **Path finding**: "How does data flow from A to B?"
- **Clustering**: Identifying communities and islands (our goal!)

### Our Graph Model

```
(File) <-[:DEFINED_IN]- (Program) -[:CALLS]-> (Program)
                            |
                            |-[:ACCESSES {action}]-> (Table)
```

**Nodes:**
- `File`: Physical source files (e.g., `CUSTOMER.RPGLE`)
- `Program`: Logical programs (e.g., `CUSTOMER`)
- `Table`: Database tables (e.g., `CUSTMAST`)

**Relationships:**
- `DEFINED_IN`: Links program to its source file
- `CALLS`: Program-to-program invocations
- `ACCESSES`: Program-to-table operations (with `action` property: READ/WRITE/UPDATE)

### Step 3.1: Connect to Neo4j

Test the database connection.

In [11]:
try:
    driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)
    
    # Test connection
    with driver.session() as session:
        result = session.run("RETURN 'Connection successful!' AS message")
        message = result.single()["message"]
        print(f"✅ {message}")
        
        # Check for required plugins
        try:
            gds_version = session.run("RETURN gds.version() AS v").single()["v"]
            print(f"✅ Graph Data Science plugin v{gds_version} detected")
        except:
            print("⚠️  GDS plugin not found - island detection will not work!")
            
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("\n💡 Troubleshooting:")
    print("   1. Check that Neo4j is running (docker ps)")
    print("   2. Verify NEO4J_URI and credentials")
    print("   3. Check firewall/network settings")

✅ Connection successful!
✅ Graph Data Science plugin v2.6.9 detected


### Step 3.2: Clear Previous Data

Start with a clean slate by removing any existing graph data.

> **Warning**: This will delete ALL data in the database!

In [12]:
if not df_deps.empty:
    try:
        with driver.session() as session:
            result = session.run("MATCH (n) RETURN count(n) AS before")
            before = result.single()["before"]
            
            session.run("MATCH (n) DETACH DELETE n")
            
            print(f"🗑️  Database cleared (removed {before} nodes)")
    except Exception as e:
        print(f"❌ Error clearing database: {e}")
else:
    print("⏭️  Skipping (no data to load)")

🗑️  Database cleared (removed 181 nodes)


### Step 3.3: Create Schema Constraints

Constraints ensure:
- Data uniqueness (no duplicate programs/tables)
- Faster query performance (automatic indexes)

In [13]:
if not df_deps.empty:
    try:
        with driver.session() as session:
            # Create unique constraints
            constraints = [
                "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Program) REQUIRE p.name IS UNIQUE",
                "CREATE CONSTRAINT IF NOT EXISTS FOR (t:Table) REQUIRE t.name IS UNIQUE",
                "CREATE CONSTRAINT IF NOT EXISTS FOR (f:File) REQUIRE f.path IS UNIQUE"
            ]
            
            for constraint in constraints:
                session.run(constraint)
            
            print("🔒 Schema constraints created:")
            print("   - Program.name (unique)")
            print("   - Table.name (unique)")
            print("   - File.path (unique)")
    except Exception as e:
        print(f"❌ Error creating constraints: {e}")
else:
    print("⏭️  Skipping (no data to load)")

🔒 Schema constraints created:
   - Program.name (unique)
   - Table.name (unique)
   - File.path (unique)


### Step 3.4: Define the Data Loader

This function translates our DataFrame into graph nodes and relationships.

**Key features:**
- Uses `MERGE` to avoid duplicates
- Accumulates line numbers for each relationship
- Uses APOC's `do.when` for conditional logic

In [14]:
def load_data_to_neo4j(tx, dataframe):
    """
    Load dependencies into Neo4j graph.
    
    Creates:
    - File nodes (physical source files)
    - Program nodes (logical programs)
    - Table nodes (database tables)
    - DEFINED_IN relationships (program -> file)
    - CALLS relationships (program -> program)
    - ACCESSES relationships (program -> table)
    """
    query = """
    UNWIND $batch AS row
    
    // 1. Create File and Program nodes
    MERGE (f:File {path: row.source_path})
    SET f.name = row.source + '.' + row.source_ext,
        f.extension = row.source_ext
    
    MERGE (p:Program {name: row.source})
    MERGE (p)-[:DEFINED_IN]->(f)
    
    // 2. Create relationships based on dependency type
    WITH p, row
    CALL apoc.do.when(
        row.type = 'CALLS',
        
        // CASE A: Program -> Program
        'MERGE (t:Program {name: row.target}) 
         MERGE (p)-[r:CALLS]->(t)
         ON CREATE SET r.lines = [row.line]
         ON MATCH SET r.lines = r.lines + row.line',
         
        // CASE B: Program -> Table
        'MERGE (t:Table {name: row.target}) 
         MERGE (p)-[r:ACCESSES {action: row.action}]->(t)
         ON CREATE SET r.lines = [row.line]
         ON MATCH SET r.lines = r.lines + row.line',
         
        {p:p, row:row}
    ) YIELD value
    RETURN count(*)
    """
    tx.run(query, batch=dataframe.to_dict('records'))

print("✅ Loader function defined")

✅ Loader function defined


### Step 3.5: Execute the Load

This may take a few seconds depending on the size of your codebase.

In [15]:
if not df_deps.empty:
    try:
        print("📤 Loading data into Neo4j...")
        
        with driver.session() as session:
            session.execute_write(load_data_to_neo4j, df_deps)
        
        print("✅ Data loaded successfully")
        
    except Exception as e:
        print(f"❌ Load failed: {e}")
        print("\n💡 Common issues:")
        print("   - APOC plugin not installed")
        print("   - Data format mismatch")
        print("   - Memory constraints")
else:
    print("⏭️  Skipping (no data to load)")

📤 Loading data into Neo4j...
✅ Data loaded successfully


### Step 3.6: Verify the Load

Check what was created in the database.

In [16]:
if not df_deps.empty:
    try:
        with driver.session() as session:
            # Count nodes by type
            result = session.run("""
                MATCH (n)
                RETURN labels(n)[0] AS Type, count(*) AS Count
                ORDER BY Count DESC
            """)
            
            print("📊 Nodes created:\n")
            for record in result:
                print(f"   {record['Type']:12} {record['Count']:5}")
            
            # Count relationships
            result = session.run("""
                MATCH ()-[r]->()
                RETURN type(r) AS Type, count(*) AS Count
                ORDER BY Count DESC
            """)
            
            print("\n📊 Relationships created:\n")
            for record in result:
                print(f"   {record['Type']:12} {record['Count']:5}")
                
    except Exception as e:
        print(f"❌ Verification failed: {e}")
else:
    print("⏭️  Skipping (no data loaded)")

📊 Nodes created:

   Program         59
   File            55
   Table           42

📊 Relationships created:

   ACCESSES        74
   DEFINED_IN      55
   CALLS           17


### Step 3.7: Sample Query

Let's run a quick query to see actual data.

In [17]:
if not df_deps.empty:
    try:
        with driver.session() as session:
            # Find a program that has both CALLS and ACCESSES
            result = session.run("""
                MATCH (p:Program)-[r]->(target)
                WITH p, type(r) AS relType, collect(DISTINCT target.name) AS targets
                WITH p, collect({type: relType, targets: targets}) AS relationships
                WHERE size(relationships) >= 2
                RETURN p.name AS Program, relationships
                LIMIT 3
            """)
            
            print("🔍 Sample programs with dependencies:\n")
            for record in result:
                print(f"Program: {record['Program']}")
                for rel in record['relationships']:
                    print(f"  {rel['type']:15} {', '.join(rel['targets'][:3])}")
                print()
                
    except Exception as e:
        print(f"❌ Query failed: {e}")
else:
    print("⏭️  Skipping (no data loaded)")

🔍 Sample programs with dependencies:

Program: LOADCUST2
  CALLS           PGM
  DEFINED_IN      LOADCUST2.CLLE

Program: LOADCUSTR
  ACCESSES        CUSTMAST, LENNONS1, CSZ_CUR
  DEFINED_IN      LOADCUSTR.SQLRPGLE

Program: MTNCUSTR
  ACCESSES        MSGCTL, STATES, CUSTMAST
  DEFINED_IN      MTNCUSTR.SQLRPGLE



---
## Phase 4: Finding Islands with Graph Data Science

Now for the exciting part: discovering isolated clusters in your codebase!

### What Are "Islands"?

An **island** (or "weakly connected component") is a group of programs and tables that:
- Are connected to each other (through calls or data access)
- Have **NO connections** to other parts of the system

### Why Islands Matter

**Large islands** = Tightly coupled subsystems (migration requires moving everything together)

**Small islands** = Independent modules (can be modernized separately)

**Many islands** = Fragmented system (may indicate poor integration or independent applications in same codebase)

### The Algorithm: Weakly Connected Components (WCC)

WCC traverses the graph ignoring relationship direction. Nodes that can reach each other (regardless of arrow direction) belong to the same component.

### Step 4.1: Create GDS Graph Projection

GDS works on an in-memory projection of our graph for performance.

In [18]:
if not df_deps.empty:
    try:
        with driver.session() as session:
            # Clean up any previous projections
            try:
                session.run("CALL gds.graph.drop('rpgSystem', false) YIELD graphName")
                print("🧹 Removed old projection")
            except:
                pass  # No previous projection
            
            # Create projection
            result = session.run("""
                CALL gds.graph.project(
                    'rpgSystem',
                    ['Program', 'Table'],
                    ['ACCESSES', 'CALLS']
                )
                YIELD nodeCount, relationshipCount
                RETURN nodeCount, relationshipCount
            """)
            
            stats = result.single()
            print(f"📊 Graph projected to memory:")
            print(f"   Nodes: {stats['nodeCount']}")
            print(f"   Relationships: {stats['relationshipCount']}")
            
    except Exception as e:
        print(f"❌ Projection failed: {e}")
else:
    print("⏭️  Skipping (no data loaded)")

🧹 Removed old projection
📊 Graph projected to memory:
   Nodes: 101
   Relationships: 91


### Step 4.2: Run WCC Algorithm

Execute the clustering algorithm.

In [19]:
if not df_deps.empty:
    try:
        print("🧮 Running Weakly Connected Components algorithm...")
        
        with driver.session() as session:
            result = session.run("""
                CALL gds.wcc.stream('rpgSystem')
                YIELD nodeId, componentId
                RETURN gds.util.asNode(nodeId).name AS Name, 
                       labels(gds.util.asNode(nodeId))[0] AS Type, 
                       componentId
                ORDER BY componentId, Type, Name
            """)
            
            df_wcc = pd.DataFrame([r.data() for r in result])
        
        print(f"✅ Algorithm complete - analyzed {len(df_wcc)} nodes")
        
    except Exception as e:
        print(f"❌ Algorithm failed: {e}")
        df_wcc = pd.DataFrame()
else:
    print("⏭️  Skipping (no data loaded)")
    df_wcc = pd.DataFrame()

🧮 Running Weakly Connected Components algorithm...
✅ Algorithm complete - analyzed 101 nodes


### Step 4.3: Analyze Island Distribution

How many islands did we find? How big are they?

In [20]:
if not df_wcc.empty:
    island_counts = df_wcc['componentId'].value_counts()
    
    print(f"🏝️  Found {len(island_counts)} distinct islands\n")
    
    print("📊 Island Size Distribution:")
    print(f"   Largest island: {island_counts.iloc[0]} components")
    print(f"   Smallest island: {island_counts.iloc[-1]} components")
    print(f"   Average size: {island_counts.mean():.1f} components")
    print(f"   Median size: {island_counts.median():.1f} components")
    
    print("\n📋 Top 10 Largest Islands:\n")
    print(island_counts.head(10))
    
else:
    print("⏭️  No analysis results")

🏝️  Found 25 distinct islands

📊 Island Size Distribution:
   Largest island: 22 components
   Smallest island: 2 components
   Average size: 4.0 components
   Median size: 2.0 components

📋 Top 10 Largest Islands:

componentId
2     22
0     14
21     7
91     7
60     5
45     4
38     3
73     3
85     3
88     3
Name: count, dtype: int64


### Step 4.4: Examine the Largest Island

Let's inspect what's in the biggest cluster.

In [21]:
if not df_wcc.empty:
    largest_id = df_wcc['componentId'].value_counts().index[0]
    largest_island = df_wcc[df_wcc['componentId'] == largest_id]
    
    print(f"🔍 Largest Island (ID: {largest_id})\n")
    print(f"Total components: {len(largest_island)}")
    
    print("\nBreakdown by type:")
    print(largest_island['Type'].value_counts())
    
    print("\nSample components:")
    display(largest_island.head(15))
    
else:
    print("⏭️  No analysis results")

🔍 Largest Island (ID: 2)

Total components: 22

Breakdown by type:
Type
Table      15
Program     7
Name: count, dtype: int64

Sample components:


,Name,Type,componentId
14,LOADCUSTR,Program,2
15,MTNCUSTR,Program,2
16,PMTCUSTR,Program,2
17,PMTSTATER,Program,2
18,SRV_MSGTR,Program,2
19,SRV_RANDT1,Program,2
20,STATEVAL,Program,2
21,CSZ_CUR,Table,2
22,CUSTMAST,Table,2
23,DUMMY,Table,2


### Step 4.5: Examine Small Islands

Small islands might be candidates for quick wins in modernization.

In [22]:
if not df_wcc.empty:
    # Find islands with 5 or fewer components
    island_sizes = df_wcc.groupby('componentId').size()
    small_islands = island_sizes[island_sizes <= 5].index
    
    print(f"🔬 Found {len(small_islands)} small islands (≤5 components)\n")
    
    if len(small_islands) > 0:
        print("Examples of small islands:")
        for island_id in list(small_islands)[:5]:
            island = df_wcc[df_wcc['componentId'] == island_id]
            print(f"\nIsland {island_id} ({len(island)} components):")
            for _, row in island.iterrows():
                print(f"  [{row['Type']:8}] {row['Name']}")
    
else:
    print("⏭️  No analysis results")

🔬 Found 21 small islands (≤5 components)

Examples of small islands:

Island 32 (2 components):
  [Program ] BTBR
  [Table   ] BTB

Island 34 (2 components):
  [Program ] BTIR
  [Table   ] BTID

Island 38 (3 components):
  [Program ] DEMOFCFC
  [Program ] PRT
  [Table   ] MYPRT

Island 40 (2 components):
  [Program ] DEMOPRTCTL
  [Table   ] QPRINT

Island 45 (4 components):
  [Program ] RCDLCKBAD
  [Program ] RCDLCKDEMO
  [Table   ] QCUSTCDT
  [Table   ] RRN


### Step 4.6: Clean Up Projection

Remove the in-memory graph to free resources.

In [23]:
if not df_wcc.empty:
    try:
        with driver.session() as session:
            session.run("CALL gds.graph.drop('rpgSystem', false) YIELD graphName")
        print("🧹 In-memory projection cleaned up")
    except Exception as e:
        print(f"Note: {e}")
else:
    print("⏭️  Nothing to clean up")

🧹 In-memory projection cleaned up


---
## Phase 5: Enriching the Graph with Island Metadata

Now we'll write the island information back into Neo4j as permanent data.

This will:
1. Tag every node with its `componentId`
2. Create `Island` nodes
3. Link all components to their island via `PART_OF` relationships

This makes it easy to query: "Show me everything in Island 5"

### Step 5.1: Write Component IDs to Nodes

Run WCC again, this time writing results to node properties.

In [24]:
if not df_wcc.empty:
    try:
        print("🔄 Writing component IDs to graph...")
        
        with driver.session() as session:
            # Create fresh projection (including Files this time)
            try:
                session.run("CALL gds.graph.drop('rpgSystem', false) YIELD graphName")
            except:
                pass
            
            session.run("""
                CALL gds.graph.project(
                    'rpgSystem',
                    ['Program', 'Table', 'File'],
                    ['ACCESSES', 'CALLS', 'DEFINED_IN']
                )
            """)
            
            # Run WCC in write mode
            result = session.run("""
                CALL gds.wcc.write('rpgSystem', { 
                    writeProperty: 'componentId' 
                })
                YIELD nodePropertiesWritten
                RETURN nodePropertiesWritten
            """).single()
            
            print(f"✅ Tagged {result['nodePropertiesWritten']} nodes with componentId")
            
    except Exception as e:
        print(f"❌ Write failed: {e}")
else:
    print("⏭️  Skipping (no analysis results)")

🔄 Writing component IDs to graph...
✅ Tagged 156 nodes with componentId


### Step 5.2: Create Island Nodes

Create a central node for each island and link all members to it.

In [25]:
if not df_wcc.empty:
    try:
        print("🏝️  Creating Island nodes...")
        
        with driver.session() as session:
            result = session.run("""
                MATCH (n) WHERE n.componentId IS NOT NULL
                
                // Create Island node
                MERGE (i:Island {island_id: n.componentId})
                
                // Link component to island
                MERGE (n)-[:PART_OF]->(i)
                
                RETURN count(DISTINCT i) AS islandsCreated
            """).single()
            
            print(f"✅ Created {result['islandsCreated']} Island nodes")
            
    except Exception as e:
        print(f"❌ Creation failed: {e}")
else:
    print("⏭️  Skipping (no analysis results)")

🏝️  Creating Island nodes...
✅ Created 25 Island nodes


### Step 5.3: Add Island Statistics

Enrich Island nodes with useful metadata.

In [26]:
if not df_wcc.empty:
    try:
        print("📊 Computing island statistics...")
        
        with driver.session() as session:
            session.run("""
                MATCH (i:Island)<-[:PART_OF]-(n)
                WITH i, 
                     count(n) AS size,
                     count(CASE WHEN 'Program' IN labels(n) THEN 1 END) AS programs,
                     count(CASE WHEN 'Table' IN labels(n) THEN 1 END) AS tables,
                     count(CASE WHEN 'File' IN labels(n) THEN 1 END) AS files
                SET i.size = size,
                    i.programs = programs,
                    i.tables = tables,
                    i.files = files
            """)
            
            print("✅ Island statistics computed")
            
    except Exception as e:
        print(f"❌ Statistics failed: {e}")
else:
    print("⏭️  Skipping (no analysis results)")

📊 Computing island statistics...
✅ Island statistics computed


### Step 5.4: Final Cleanup

Remove the GDS projection one last time.

In [27]:
if not df_wcc.empty:
    try:
        with driver.session() as session:
            session.run("CALL gds.graph.drop('rpgSystem', false) YIELD graphName")
        print("🧹 Final cleanup complete")
    except:
        pass
else:
    print("⏭️  Nothing to clean up")

🧹 Final cleanup complete


---
## Phase 6: Verification and Exploration

Let's verify our enriched graph and explore some useful queries.

### Step 6.1: Verify Island Structure

Check that islands are properly isolated by looking for any cross-island dependencies.

In [28]:
if not df_wcc.empty:
    try:
        with driver.session() as session:
            # Check for any connections between different islands
            result = session.run("""
                MATCH (n1)-[:PART_OF]->(i1:Island),
                      (n2)-[:PART_OF]->(i2:Island)
                WHERE i1.id <> i2.id
                  AND (n1)-[:CALLS|ACCESSES]-(n2)
                RETURN n1.name AS Source, 
                       i1.id AS SourceIsland,
                       n2.name AS Target,
                       i2.id AS TargetIsland
                LIMIT 10
            """)
            
            cross_island = pd.DataFrame([r.data() for r in result])
            
            if cross_island.empty:
                print("✅ Verification passed: Islands are properly isolated")
            else:
                print("⚠️  Found cross-island dependencies (unexpected):")
                display(cross_island)
                
    except Exception as e:
        print(f"❌ Verification failed: {e}")
else:
    print("⏭️ Skipping (no data available)")

Received notification from DBMS server: <GqlStatusObject gql_status='01N42', status_description="One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: id)", position=<SummaryInputPosition line=4, column=26, offset=129>, raw_classification='UNRECOGNIZED', classification=<NotificationClassification.UNRECOGNIZED: 'UNRECOGNIZED'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/', '_classification': 'UNRECOGNIZED', '_severity': 'WARNING', '_position': {'column': 26, 'offset': 129, 'line': 4}}> for query: '\n                MATCH (n1)-[:PART_OF]->(i1:Island),\n                      (n2)-[:PART_OF]->(i2:Island)\n                WHERE i1.id <> i2.id\n                  AND (n1)-[:CALLS|ACCESSES]-(n2)\n                RETURN n1.n

✅ Verification passed: Islands are properly isolated


### Step 6.2: Explore Island Contents

Now let's examine specific islands to understand their structure.

In [29]:
if not df_wcc.empty:
    try:
        with driver.session() as session:
            # Pick an island to visualize (use the largest one)
            result = session.run("""
                MATCH (i:Island)
                WITH i ORDER BY i.size DESC LIMIT 1
                
                MATCH (i)<-[:PART_OF]-(n)
                OPTIONAL MATCH (n)-[r:CALLS|ACCESSES]->(m)
                RETURN n.name AS Component,
                       labels(n)[0] AS Type,
                       type(r) AS RelType,
                       m.name AS Target
                ORDER BY Type, Component
                LIMIT 50
            """)
            
            df_island = pd.DataFrame([r.data() for r in result])
            
            if not df_island.empty:
                print("🔍 Sample Island Contents:\n")
                display(df_island.head(20))
            else:
                print("No island data found")
                
    except Exception as e:
        print(f"❌ Query failed: {e}")
else:
    print("⏭️ Skipping (no data available)")

🔍 Sample Island Contents:



,Component,Type,RelType,Target
0,LOADCUSTR.SQLRPGLE,File,NaN,NaN
1,MTNCUSTR.SQLRPGLE,File,NaN,NaN
2,MTNCUSTR.SQLRPGLE,File,NaN,NaN
3,PMTCUSTR.SQLRPGLE,File,NaN,NaN
4,PMTSTATER.SQLRPGLE,File,NaN,NaN
5,SRV_MSGTR.RPGLE,File,NaN,NaN
6,SRV_RANDT1.SQLRPGLE,File,NaN,NaN
7,STATEVAL.SQLRPGLE,File,NaN,NaN
8,LOADCUSTR,Program,ACCESSES,LENNONS1
9,LOADCUSTR,Program,ACCESSES,CSZ_CUR


### Step 6.3: Sample Queries for Exploration

Here are some useful queries to explore the dependency graph. These can be run in this notebook or in Neo4j Browser (`http://localhost:7474`).

### Visual Exploration in Neo4j Browser

For interactive graph visualization, open Neo4j Browser at `http://localhost:7474` and try:

**View a specific island visually:**
```cypher
MATCH (i:Island {island_id: 5})<-[:PART_OF]-(n)
OPTIONAL MATCH (n)-[r:CALLS|ACCESSES]->(m)
RETURN i, n, r, m
LIMIT 50
```

**Tips for Neo4j Browser:**
- Click nodes to see properties
- Use different colors for different node types
- Drag nodes to rearrange the layout
- Double-click to expand relationships
- Use the graph style editor to customize appearance

In [30]:
if not df_wcc.empty:
    try:
        with driver.session() as session:
            # First check if MTNCUSTR exists, otherwise use first available program
            check = session.run("MATCH (p:Program) RETURN p.name LIMIT 1").single()
            
            if check:
                # Try MTNCUSTR first, fallback to any program
                program_name = 'MTNCUSTR'
                exists = session.run("MATCH (p:Program {name: $name}) RETURN p", name=program_name).single()
                
                if not exists:
                    program_name = check['p.name']
                    print(f"ℹ️  MTNCUSTR not found, using {program_name} instead\n")
                
                result = session.run("""
                    MATCH path = (p:Program {name: $name})-[:CALLS|ACCESSES*1..3]->(target)
                    RETURN p.name AS StartProgram,
                           [node in nodes(path) | node.name] AS Path,
                           length(path) AS Depth
                    ORDER BY Depth, Path
                    LIMIT 25
                """, name=program_name)
                
                df_trace = pd.DataFrame([r.data() for r in result])
                
                if not df_trace.empty:
                    print(f"🔗 Dependency trace from {program_name}:\n")
                    display(df_trace)
                    
                    print(f"\n📊 Depth distribution:")
                    print(df_trace['Depth'].value_counts().sort_index())
                else:
                    print(f"No dependencies found for {program_name}")
            else:
                print("No programs found in database")
                
    except Exception as e:
        print(f"❌ Query failed: {e}")
else:
    print("⏭️ Skipping (no data available)")

🔗 Dependency trace from MTNCUSTR:



,StartProgram,Path,Depth
0,MTNCUSTR,"[MTNCUSTR, CUSTMAST]",1
1,MTNCUSTR,"[MTNCUSTR, MSGCTL]",1
2,MTNCUSTR,"[MTNCUSTR, MTNCUSTD]",1
3,MTNCUSTR,"[MTNCUSTR, SFT_FKEY]",1
4,MTNCUSTR,"[MTNCUSTR, SH_HDR]",1
5,MTNCUSTR,"[MTNCUSTR, STATES]",1



📊 Depth distribution:
Depth
1    6
Name: count, dtype: int64


#### Query 1: Visualize a Specific Island

Show all components and their relationships within a specific island (Island 1 in this example).

In [31]:
with driver.session() as session:
    result = session.run("""
        MATCH (i:Island {island_id: 1})<-[:PART_OF]-(n)
        OPTIONAL MATCH (n)-[r:CALLS|ACCESSES]->(m)
        RETURN n.name AS Component,
               labels(n)[0] AS Type,
               type(r) AS RelType,
               m.name AS Target
        ORDER BY Type, Component
        LIMIT 50
    """)
    df_island = pd.DataFrame([r.data() for r in result])

df_island.head()

,Component,Type,RelType,Target
0,LOADCUSTR.SQLRPGLE,File,NaN,NaN
1,MTNCUSTR.SQLRPGLE,File,NaN,NaN
2,MTNCUSTR.SQLRPGLE,File,NaN,NaN
3,PMTCUSTR.SQLRPGLE,File,NaN,NaN
4,PMTSTATER.SQLRPGLE,File,NaN,NaN


#### Query 2: Most-Called Programs

Find programs that are called most frequently by other programs.

In [32]:
with driver.session() as session:
    result = session.run("""
        MATCH (caller:Program)-[r:CALLS]->(callee:Program)
        RETURN callee.name AS Program, count(r) AS CallCount
        ORDER BY CallCount DESC
        LIMIT 10
    """)
    df_calls = pd.DataFrame([r.data() for r in result])

df_calls

,Program,CallCount
0,PGM,9
1,RPG,1
2,MESSAGE,1
3,T9DSPMNY,1
4,T9DSPMAY,1
5,EXCCMD,1
6,PROCPTR,1
7,PROC1,1
8,PROC2,1


#### Query 3: Most-Accessed Tables

Find database tables that are used by the most programs.

In [33]:
with driver.session() as session:
    result = session.run("""
        MATCH (p:Program)-[r:ACCESSES]->(t:Table)
        RETURN t.name AS Table, count(DISTINCT p) AS UsedByPrograms
        ORDER BY UsedByPrograms DESC
        LIMIT 10
    """)
    df_tables = pd.DataFrame([r.data() for r in result])

df_tables

,Table,UsedByPrograms
0,MYFILE,5
1,SFT_FKEY,4
2,MSGCTL,4
3,SH_HDR,4
4,QIWS,4
5,STATES,3
6,CUSTMAST,3
7,SFL,2
8,LENNONS1,2
9,SFLCTL,2


#### Query 4: Programs with READ and WRITE to Same Table

Find programs that both read from and write to the same table (potential data transformation logic).

In [34]:
with driver.session() as session:
    result = session.run("""
        MATCH (p:Program)-[r1:ACCESSES {action: 'READ'}]->(t:Table),
              (p)-[r2:ACCESSES {action: 'WRITE'}]->(t)
        RETURN p.name AS Program, t.name AS Table
        ORDER BY Program, Table
        LIMIT 10
    """)
    df_rw = pd.DataFrame([r.data() for r in result])

df_rw

,Program,Table
0,DEMOFCFC,MYPRT
1,DEMOPRTCTL,QPRINT
2,PRT,MYPRT
3,USADRVAL_T,QSYSPRT


#### Query 5: All Islands Summary

List all islands with their metadata (size, programs, tables, files).

In [35]:
with driver.session() as session:
    result = session.run("""
        MATCH (i:Island)
        RETURN i.island_id AS IslandID,
               i.size AS Size,
               i.programs AS Programs,
               i.tables AS Tables,
               i.files AS Files
        ORDER BY i.size DESC
    """)
    df_all_islands = pd.DataFrame([r.data() for r in result])

df_all_islands.head()

,IslandID,Size,Programs,Tables,Files
0,1,30,7,15,8
1,0,26,14,0,12
2,48,12,5,2,5
3,6,10,2,5,3
4,31,9,4,1,4


#### Query 6: Orphan Programs

Find programs that have no dependencies (don't call anything and don't access any tables).

In [36]:
with driver.session() as session:
    result = session.run("""
        MATCH (p:Program)
        WHERE NOT (p)-[:CALLS|ACCESSES]->()
        RETURN p.name AS OrphanProgram
        ORDER BY OrphanProgram
    """)
    df_orphans = pd.DataFrame([r.data() for r in result])

df_orphans

,OrphanProgram
0,EXCCMD
1,MESSAGE
2,PGM
3,PROC1
4,PROC2
5,PROCPTR
6,RPG
7,T9DSPMAY
8,T9DSPMNY


#### Query 7: Trace Program Dependencies

Trace the dependency path from a specific program (example: MTNCUSTR) up to 3 levels deep.

In [37]:
with driver.session() as session:
    result = session.run("""
        MATCH path = (start:Program {name: 'MTNCUSTR'})-[:CALLS|ACCESSES*1..3]->()
        RETURN [node in nodes(path) | node.name] AS DependencyPath
        LIMIT 20
    """)
    df_trace = pd.DataFrame([r.data() for r in result])

df_trace

,DependencyPath
0,"[MTNCUSTR, MTNCUSTD]"
1,"[MTNCUSTR, SH_HDR]"
2,"[MTNCUSTR, SFT_FKEY]"
3,"[MTNCUSTR, CUSTMAST]"
4,"[MTNCUSTR, STATES]"
5,"[MTNCUSTR, MSGCTL]"


---
## Phase 7: LLM-Powered Island Description

Use DeepSeek 3.2 to analyze source code in each island and generate:
- A descriptive name for the island
- A summary of what the island does

This helps identify the purpose and functionality of each isolated component cluster.

### Step 7.1: Set Up DeepSeek API Connection

We'll use the OpenAI SDK with DeepSeek's API endpoint. Make sure you have:
- `DEEPSEEK_API_KEY` environment variable set
- `DEEPSEEK_BASE_URL` environment variable (optional, defaults to DeepSeek's API)

In [38]:
import os
from openai import OpenAI

# Set up DeepSeek API client
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
deepseek_base_url = os.getenv('DEEPSEEK_BASE_URL', 'https://api.deepseek.com')

if not deepseek_api_key:
    print("⚠️ Warning: DEEPSEEK_API_KEY not found in environment variables")
    print("Set it with: export DEEPSEEK_API_KEY='your-key-here'")
else:
    client = OpenAI(
        api_key=deepseek_api_key,
        base_url=deepseek_base_url
    )
    print("✅ DeepSeek API client initialized")
    print(f"   Base URL: {deepseek_base_url}")

✅ DeepSeek API client initialized
   Base URL: https://api.deepseek.com


### Step 7.2: Get Islands and Their Programs

Query Neo4j to get all islands and the programs that belong to each island.

In [39]:
with driver.session() as session:
    result = session.run("""
        MATCH (i:Island)<-[:PART_OF]-(p:Program)-[:DEFINED_IN]->(f:File)
        WITH i, collect(p.name) AS programs, collect(f.path) AS file_paths
        RETURN 
            i.island_id AS island_id,
            i.size AS size,
            programs,
            file_paths
        ORDER BY i.size DESC
    """)
    df_islands_for_analysis = pd.DataFrame([r.data() for r in result])

df_islands_for_analysis.head()

,island_id,size,programs,file_paths
0,1,30,"[LOADCUSTR, MTNCUSTR, PMTCUSTR, PMTSTATER, SRV...",[src/IBM-i-RPG-Free-CLP-Code/5250_Subfile/LOAD...
1,0,26,"[LOADCUST2, GETOBJUC, LCKOBJC, T9ALLOCMNY, GET...",[src/IBM-i-RPG-Free-CLP-Code/5250_Subfile/LOAD...
2,48,12,"[DELETE, READ, READE, READP, WRITE]",[src/rpgleparser/src/test/resources/org/rpglep...
3,6,10,"[GETOBJUR, GETOBJUR, DEMO_RPG1]",[src/IBM-i-RPG-Free-CLP-Code/APIs/GETOBJUR.RPG...
4,31,9,"[SQL_SKEL, SQL_SKEL2, SQL_SKEL3, SQL_SKELNF]",[src/IBM-i-RPG-Free-CLP-Code/SQL_SKELETON/sql_...


### Step 7.3: Load Source Code for an Island

Helper function to load all source code files for a given island.

In [40]:
def load_island_source_code(file_paths, max_files=10):
    """
    Load source code from disk for files in an island.
    
    Args:
        file_paths: List of file paths
        max_files: Maximum number of files to load (to avoid token limits)
    
    Returns:
        Dictionary mapping filename to source code
    """
    source_files = {}
    
    for file_path in file_paths[:max_files]:
        if os.path.exists(file_path):
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read()
                    filename = os.path.basename(file_path)
                    source_files[filename] = content
            except Exception as e:
                print(f"⚠️ Could not read {file_path}: {e}")
        else:
            print(f"⚠️ File not found: {file_path}")
    
    return source_files

print("✅ Helper function defined")

✅ Helper function defined


### Step 7.4: Analyze Islands with DeepSeek

For each island, send the source code to DeepSeek and ask it to:
1. Generate a descriptive name
2. Provide a summary of functionality

In [41]:
def analyze_island_with_ai(island_id, source_files, client):
    """
    Use DeepSeek API to analyze an island's source code.
    
    Args:
        island_id: The island ID
        source_files: Dictionary of filename -> source code
        client: OpenAI client configured for DeepSeek
    
    Returns:
        Dictionary with 'name' and 'summary'
    """
    # Create prompt with source code
    prompt = f"""You are analyzing a cluster of legacy RPG programs (Island {island_id}) on IBM i (AS/400).

Below are the source code files in this cluster:

"""
    
    for filename, content in source_files.items():
        # Truncate very long files
        truncated_content = content[:5000] if len(content) > 5000 else content
        prompt += f"\n### File: {filename}\n```rpg\n{truncated_content}\n```\n"
    
    prompt += """

Based on this source code, provide:
1. A short descriptive name for this island (2-5 words)
2. A summary of what this island does (2-3 sentences)

Format your response as:
NAME: [your name here]
SUMMARY: [your summary here]
"""
    
    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "You are an expert in legacy IBM i RPG code analysis."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1,
            max_tokens=500
        )
        
        content = response.choices[0].message.content
        
        # Parse response
        lines = content.strip().split('\n')
        name = ""
        summary = ""
        
        for line in lines:
            if line.startswith("NAME:"):
                name = line.replace("NAME:", "").strip()
            elif line.startswith("SUMMARY:"):
                summary = line.replace("SUMMARY:", "").strip()
        
        return {
            "island_id": island_id,
            "ai_name": name,
            "ai_summary": summary
        }
    
    except Exception as e:
        print(f"❌ Error analyzing island {island_id}: {e}")
        return {
            "island_id": island_id,
            "ai_name": "Error",
            "ai_summary": str(e)
        }

print("✅ Analysis function defined")

✅ Analysis function defined


### Step 7.5: Run Analysis on All Islands

Process each island and collect the AI-generated names and summaries.

⚠️ **Note:** This will make API calls to DeepSeek for each island. Consider running on a subset first.

In [42]:
# Analyze all islands
island_analyses = []

for idx, row in df_islands_for_analysis.iterrows():
    island_id = row['island_id']
    file_paths = row['file_paths']
    
    print(f"\n{'='*60}")
    print(f"Analyzing Island {island_id} ({len(file_paths)} programs)...")
    print(f"{'='*60}")
    
    # Load source code
    source_files = load_island_source_code(file_paths, max_files=5)
    print(f"✅ Loaded {len(source_files)} source files")
    
    # Analyze with AI
    if source_files and deepseek_api_key:
        analysis = analyze_island_with_ai(island_id, source_files, client)
        island_analyses.append(analysis)
        
        print(f"\n📝 AI Analysis:")
        print(f"   Name: {analysis['ai_name']}")
        print(f"   Summary: {analysis['ai_summary']}")
    else:
        print("⚠️ Skipping AI analysis (no source files or API key not set)")

# Convert to DataFrame
df_island_analyses = pd.DataFrame(island_analyses)

df_island_analyses.head()


Analyzing Island 1 (8 programs)...
✅ Loaded 5 source files

📝 AI Analysis:
   Name: Customer Management System
   Summary: This island contains a suite of RPG programs for managing customer master data on IBM i. It includes programs for generating test customer data, maintaining customer records through interactive screens, searching/prompting for customers, and providing state code lookups. The system uses SQL for database operations and follows modern RPG free-format coding standards.

Analyzing Island 0 (12 programs)...
✅ Loaded 4 source files

📝 AI Analysis:
   Name: Object Lock Management Utilities
   Summary: This island contains CL programs for managing object locks and allocations on IBM i. It includes utilities to check which jobs are using an object, attempt to lock objects with operator intervention on conflicts, and submit batch jobs for loading test data. The programs handle scenarios like exclusive locks, wait times, and user notifications.

Analyzing Island 48 (5 progra

,island_id,ai_name,ai_summary
0,1,Customer Management System,This island contains a suite of RPG programs f...
1,0,Object Lock Management Utilities,This island contains CL programs for managing ...
2,48,File Operation Examples,This island contains a set of RPG programs tha...
3,6,Object Lock Analysis Utilities,This island contains programs that identify wh...
4,31,SQL Skeleton Program Cluster,This island contains multiple versions of a te...


### Step 7.6: Update Neo4j with AI Analysis (Optional)

Store the AI-generated names and summaries back in Neo4j on the Island nodes.

In [43]:
with driver.session() as session:
    for _, row in df_island_analyses.iterrows():
        session.run("""
            MATCH (i:Island {island_id: $island_id})
            SET i.ai_name = $ai_name,
                i.ai_summary = $ai_summary
        """, 
        island_id=row['island_id'],
        ai_name=row['ai_name'],
        ai_summary=row['ai_summary']
        )

print(f"✅ Updated {len(df_island_analyses)} islands with AI analysis")

✅ Updated 25 islands with AI analysis


### Step 7.7: View All Island Names and Summaries

Final summary of all discovered islands with their AI-generated names and descriptions.

In [44]:
with driver.session() as session:
    result = session.run("""
        MATCH (i:Island)
        RETURN 
            i.island_id AS IslandID,
            i.size AS Size,
            i.ai_name AS Name,
            i.ai_summary AS Summary
        ORDER BY i.size DESC
    """)
    df_island_summary = pd.DataFrame([r.data() for r in result])

df_island_summary

,IslandID,Size,Name,Summary
0,1,30,Customer Management System,This island contains a suite of RPG programs f...
1,0,26,Object Lock Management Utilities,This island contains CL programs for managing ...
2,48,12,File Operation Examples,This island contains a set of RPG programs tha...
3,6,10,Object Lock Analysis Utilities,This island contains programs that identify wh...
4,31,9,SQL Skeleton Program Cluster,This island contains multiple versions of a te...
5,22,6,Record Lock Demonstration Programs,This island contains two RPG programs that dem...
6,17,5,Printer Control Utility Island,This island contains RPG programs that manage ...
7,40,5,Booking Data SQL Cursor,This island defines an SQL cursor to retrieve ...
8,46,4,Program Call Handler,This island orchestrates calls to various proc...
9,47,4,CHAIN Operation Island,This island contains RPG code focused on perfo...
